In [1]:
import requests
from pymongo import MongoClient
from urllib.parse import urljoin
from pprint import pprint
import nltk
import json

# Retrieve Lyrics Via API

In [10]:
# DB CONFIG
client = MongoClient('mongo', 27017)
db = client.music_db
COLLECTION = 'rap-song'
songs_collection = db[COLLECTION]

In [11]:
# Pull all songs from DB
songs = list(songs_collection.find())

In [18]:
MUSICMATCH_API = '309401c7a9740d7c3bfbc94bca9a911c'
APISEED_API = 'qD6PYe4ekCWJkwSYLiG5JI7DB3Up3kGF7kfiAKXGqwNh519WjfghnFhZJfReyE3B'

In [20]:
class ApiSeed:
    BASE_URL = 'https://orion.apiseeds.com/api/music/lyric/'
    
    def __init__(self, api_key):
        """ Load automatically translates responses from HTTP request objects into Dictionaries """
        self.api_key = api_key
        self.load = True
    
    def _api_url(self, url):
        url = urljoin(self.BASE_URL, url)
        url = urljoin(url, f'?apikey={APISEED_API}')
        return url
    
    def _retrieve(self, url):
        response = requests.get(self._api_url(url))
        if self.load:
            response = json.loads(response.content)
        return response
    
    def get_track(self, artist, track):
        return self._retrieve(f'{artist}/{track}')
    

apiseed = ApiSeed(APISEED_API)
res = apiseed.get_track('Kanye West', 'Everything I am')

# Insert Lyrics into DB

In [21]:
import time
def insert_into_db():
    for song in songs:
        pass
    
def update_db(document_id, lyrics: str):
    document = songs_collection.find_one_and_update({"_id": document_id}, 
                                         {"$set": {"lyrics": lyrics}})
    return document

def retrieve_lyrics(track: dict):
    """ Takes mongodb document and gets lyrics"""
    res = apiseed.get_track(track['artist'], track['title'])
    return res

def find_and_update_lyrics(track: dict):
    """ Updates track with lyrics """
    lyrics = retrieve_lyrics(track)
    track['lyrics'] = lyrics
    return track

        
start_index = 0
all_tracks = []
for i, track in enumerate(songs[start_index:], start_index):
    try:
        track = find_and_update_lyrics(track)

        if track.get('lyrics').get('error'):
            print("Error", track.get('lyrics').get('error'), end=" "); 
        else:
            print("Success", end=" ")
            
    except (json.JSONDecodeError, Exception) as e:
        print(e)
        track['lyrics'] = 'null'
        
    update_db(track['_id'], track['lyrics'])
    print(i, track['title'], end = " | ")
    time.sleep(.33)



Success 0 In My Feelings | Success 1 Killshot | Success 2 Lucid Dreams | Success 3 Better Now | Error Lyric no found, try again later. 4 I Like It | Error Lyric no found, try again later. 5 I Love It | Error Lyric no found, try again later. 6 FEFE | Error Lyric no found, try again later. 7 Sicko Mode | Error Lyric no found, try again later. 8 Taste | Error Lyric no found, try again later. 9 Lucky You | Success 10 Nonstop | Error Lyric no found, try again later. 11 Yes Indeed | Success 12 Rap Devil | Error Lyric no found, try again later. 13 Big Bank | Error Lyric no found, try again later. 14 Drip Too Hard | Error Lyric no found, try again later. 15 Wake Up In The Sky | Error Lyric no found, try again later. 16 Nice For What | Success 17 New Patek | Success 18 Mo Bamba | Error Lyric no found, try again later. 19 Fall | Success 20 Moonlight | Error Lyric no found, try again later. 21 Ring | Success 22 Barbie Dreams | Success 23 The Ringer | Error Lyric no found, try again later. 24 Smil

In [22]:
songs_collection = db[COLLECTION]

def has_lyrics(song):
    if not song.get('lyrics'):
        return False
    
    if song.get('lyrics') == 'null':
        return False

    if song.get('lyrics').get('error'):
        return False
        
    return True


In [8]:
songs = [song for song in songs_collection.find() if has_lyrics(song)]

In [9]:
len(songs)

0